In [6]:
using Plots
#Error functions
function fidelity(ρ,σ)
    ##make sure ρ and σ are in matrix form
    matrix = sqrt(ρ)*σ*sqrt(ρ)
    matrix = sqrt(matrix)
    fid = tr(matrix)*conj(tr(matrix))
    return fid
end

function trace_dist(ρ,σ)
    matrix = (ρ-σ)*(ρ-σ)'
    matrix = sqrt(matrix)
    return 0.5*tr(matrix)
end

function error_propagation(corr,times,H_single,δt,Ci,N,Krylov_cutoff,k2,cutoff,other_norms,save_bool)
    parameter_string = "N="*string(N)*", δt="*string(δt)*", Krylov cutoff="*string(Krylov_cutoff)*
            ", k2="*string(k2)*", truncation cutoff="*string(cutoff)
    corr_list = copy(corr)
    time_list = copy(times)
    sim_ρ_list = corr_list./tr.(corr_list)
    exact_corr_list = Any[]
    exact_ρ_list = Any[]
    fid_list = Any[]
    trace_list = Any[]
    n = length(times)
    
    pushfirst!(corr_list,Ci)
    pushfirst!(time_list,0)
    pushfirst!(exact_corr_list,Ci)
    pushfirst!(exact_ρ_list,Ci/tr(Ci))
    pushfirst!(sim_ρ_list,Ci/tr(Ci))
    pushfirst!(fid_list,1) 
    pushfirst!(trace_list,0)
    U_step= exp(-im*δt*H_single)
    
    for i =1:n
        corr_term = U_step*(exact_corr_list[i])*U_step'
        ρ_term = corr_term/tr(corr_term)
        fid_term = fidelity(ρ_term,sim_ρ_list[i+1])
        trace_term = abs(trace_dist(ρ_term,sim_ρ_list[i+1]))
        
        push!(exact_corr_list,corr_term)
        push!(exact_ρ_list,ρ_term)
        push!(fid_list,fid_term)
        push!(trace_list,trace_term)
    end
    
    fid_list = fid_list .- 1 ### The fidelity of two identical density matrices is 1
#     println(abs.(trace_list[1]))
#     println(abs.(trace_list[2]))
#     println(abs.(trace_list[5]))
#     println(abs.(trace_list[10]))
#     println(abs.(trace_list[20]))
#     println(abs.(trace_list[30]))
#     println(abs.(trace_list[40]))
#     println(abs.(trace_list[50]))
    
    
    
    
    diff_list = exact_corr_list - corr_list
    perc_diff_norm1_list = opnorm.(diff_list,1)./(opnorm.(exact_corr_list,1))
    perc_diff_norm2_list = opnorm.(diff_list,2)./(opnorm.(exact_corr_list,2))
    perc_diff_norminf_list = opnorm.(diff_list,Inf)./(opnorm.(exact_corr_list,Inf))
    perc_diff_normfrob_list = norm.(diff_list)./(norm.(exact_corr_list,1))
    max_diff_list = maximum.([abs.(diff_list[i]) for i=1:(size(diff_list)[1])]) 
   if other_norms
        plot(time_list,(abs.(perc_diff_norm1_list)),label="p=1 matrix norm")
        plot!(time_list,(abs.(perc_diff_norm2_list)),;label="p=2 matrix norm")
        plot!(time_list,(abs.(perc_diff_norminf_list)),label="p=inf matrix norm")
        plot!(time_list,(abs.(perc_diff_normfrob_list)),label="frobenius norm")
        plot!(time_list,(abs.(fid_list)),label="fidelity - 1")
        display(plot!(time_list,(abs.(trace_list)),label="trace distance,"*parameter_string))
   else
        display(plot(time_list,(abs.(trace_list)),label="trace distance,"*parameter_string))
   end

   if other_norms
        plot(time_list,log10.(abs.(perc_diff_norm1_list)),label="p=1 matrix norm")
        plot!(time_list,log10.(abs.(perc_diff_norm2_list)),;label="p=2 matrix norm")
        plot!(time_list,log10.(abs.(perc_diff_norminf_list)),label="p=inf matrix norm")
        plot!(time_list,log10.(abs.(perc_diff_normfrob_list)),label="frobenius norm")
        plot!(time_list,log10.(abs.(fid_list)),label="fidelity - 1")
        display(plot!(time_list,log10.(abs.(trace_list)),
           label="trace distance,"*parameter_string))
    else
        display(plot(time_list,log10.(abs.(trace_list)),
           label="trace distance,"*parameter_string))
    end
    
    if save_bool
        savefig("error propagation,"*parameter_string*".png") 
    end
    end


function error_propagation2(corr,times1,times2,H_single,δt1,δt2,Ci,N,Krylov_cutoff,k2,cutoff,other_norms,save_bool)
    parameter_string = "N="*string(N)*", δt="*string(δt)*", Krylov cutoff="*string(Krylov_cutoff)*
            ", k2="*string(k2)*", truncation cutoff="*string(cutoff)
    corr_list = copy(corr)
    t1 = deepcopy(times1)
    t2 = deepcopy(times2)
    n1 = length(t1)
    n2 = length(t2)
    
    time_list = append!(t1,t2)
    sim_ρ_list = corr_list./tr.(corr_list)
    exact_corr_list = Any[]
    exact_ρ_list = Any[]
    fid_list = Any[]
    trace_list = Any[]

    pushfirst!(corr_list,Ci)
    pushfirst!(time_list,0)
    pushfirst!(exact_corr_list,Ci)
    pushfirst!(exact_ρ_list,Ci/tr(Ci))
    pushfirst!(sim_ρ_list,Ci/tr(Ci))
    pushfirst!(fid_list,1) 
    pushfirst!(trace_list,0)
    
    U_step1= exp(-im*δt1*H_single)
    U_step2 = exp(-im*δt2*H_single)
 
    for i =1:n1
        corr_term = U_step1*last(exact_corr_list)*U_step1'
        ρ_term = corr_term/tr(corr_term)
        fid_term = fidelity(ρ_term,sim_ρ_list[i+1])
        trace_term = abs(trace_dist(ρ_term,sim_ρ_list[i+1]))
        
        push!(exact_corr_list,corr_term)
        push!(exact_ρ_list,ρ_term)
        push!(fid_list,fid_term)
        push!(trace_list,trace_term)
    end
    
    for i = (n1+1):n1+n2
        corr_term = U_step2*last(exact_corr_list)*U_step2'
        ρ_term = corr_term/tr(corr_term)
        fid_term = fidelity(ρ_term,sim_ρ_list[i+1])
        trace_term = abs(trace_dist(ρ_term,sim_ρ_list[i+1]))
        
        push!(exact_corr_list,corr_term)
        push!(exact_ρ_list,ρ_term)
        push!(fid_list,fid_term)
        push!(trace_list,trace_term)
    end
    
    fid_list = fid_list .- 1 ### The fidelity of two identical density matrices is 1
 
    diff_list = exact_corr_list - corr_list
    perc_diff_norm1_list = opnorm.(diff_list,1)./(opnorm.(exact_corr_list,1))
    perc_diff_norm2_list = opnorm.(diff_list,2)./(opnorm.(exact_corr_list,2))
    perc_diff_norminf_list = opnorm.(diff_list,Inf)./(opnorm.(exact_corr_list,Inf))
    perc_diff_normfrob_list = norm.(diff_list)./(norm.(exact_corr_list,1))
    max_diff_list = maximum.([abs.(diff_list[i]) for i=1:(size(diff_list)[1])]) 
   if other_norms
        plot(time_list,(abs.(perc_diff_norm1_list)),label="p=1 matrix norm")
        plot!(time_list,(abs.(perc_diff_norm2_list)),;label="p=2 matrix norm")
        plot!(time_list,(abs.(perc_diff_norminf_list)),label="p=inf matrix norm")
        plot!(time_list,(abs.(perc_diff_normfrob_list)),label="frobenius norm")
        plot!(time_list,(abs.(fid_list)),label="fidelity - 1")
        display(plot!(time_list,(abs.(trace_list)),label="trace distance,"*parameter_string))
   else
        display(plot(time_list,(abs.(trace_list)),label="trace distance,"*parameter_string))
   end

   if other_norms
        plot(time_list,log10.(abs.(perc_diff_norm1_list)),label="p=1 matrix norm")
        plot!(time_list,log10.(abs.(perc_diff_norm2_list)),;label="p=2 matrix norm")
        plot!(time_list,log10.(abs.(perc_diff_norminf_list)),label="p=inf matrix norm")
        plot!(time_list,log10.(abs.(perc_diff_normfrob_list)),label="frobenius norm")
        plot!(time_list,log10.(abs.(fid_list)),label="fidelity - 1")
        display(plot!(time_list,log10.(abs.(trace_list)),
           label="trace distance,"*parameter_string))
    else
        display(plot(time_list,log10.(abs.(trace_list)),
           label="trace distance,"*parameter_string))
    end

    if save_bool
        savefig("error propagation,"*parameter_string*".png") 
    end
   
end


function correlation_matrix_statistics(Cf,Cf_test,eps)
   
#     trun_ind1 = findall(x -> (abs(x)>eps), Cf)
#     trun_ind2 = findall(x -> (abs(x)>eps), Cf_test)
#     bool = size(trun_ind1)[1]>size(trun_ind2)[1]
#     if bool
#         trun_ind = trun_ind1
#     else
#         trun_ind = trun_ind2
#     end
#     Cf_trun = Cf[trun_ind]
#     Cf_test_trun = Cf_test[trun_ind]
#     Cf_trun_diff = Cf_trun-Cf_test_trun
    Cf_trun_diff = Cf-Cf_test
    
    norm0 = opnorm(Cf,1)
    norm1 = opnorm(Cf_trun_diff,1)
    norm2 = opnorm(Cf_trun_diff,2)
    norminf = opnorm(Cf_trun_diff,Inf)
    normfrob = norm(Cf_trun_diff)
    
    println("p=1 norm of simulated correlation matrix="*string(norm0))
    println("p=1 norm of truncated difference matrix="*string(norm1))
    println("p=2 norm of truncated difference matrix="*string(norm2))
    println("p=inf norm of truncated difference matrix="*string(norminf))
    println("Frobenius norm of truncated difference matrix="*string(normfrob))
#     println("mean of truncated Cf="*string(mean(broadcast(abs,Cf_trun))))
#     println("mean of truncated Cf_test="*string(mean(broadcast(abs,Cf_test_trun))))
    println("mean of difference ="*string(mean(broadcast(abs,Cf_trun_diff))))
    
    f(x, y) = abs(Cf[x,y])

    display(surface(1:6,1:6,f,title = "Simulated Correlation matrix",
    xlabel = "sites",
    ylabel = "sites"))
    
    h(x,y) =abs(Cf_test[x,y])
    
    display(surface(1:6,1:6,h,title = "Exact correlation matrix",
    xlabel = "sites",
    ylabel = "sites"))
    
    g(x, y) = abs(Cf[x,y]-Cf_test[x,y])

    display(surface(1:6,1:6,g,title = "Difference between simulated and exact",
    xlabel = "sites",
    ylabel = "sites"))
    
    return #Cf_trun_diff

end


###Animation functions


function correlation_heatmap(corr, nframe, site_lim,T,model)
    x=y=1:site_lim
    anim = @animate for i=1:n
        f(x, y) = abs((corr[i])[x,y])
        heatmap(x,y,f,size=(400,400),aspect_ratio=:equal,clims=(0, 1),legend=true,c=:Set1_6)
    end
    gif(anim,"correlation_heatmap_anim,T="*string(T)*",model="*string(model)*".gif")
end

function correlation_contour(corr, nframe, site_lim,T,model)
    x=y=1:site_lim
    anim = @animate for i=1:n
        f(x, y) = abs((corr[i])[x,y])
        contour(x,y,f,size=(400,400),aspect_ratio=:equal,clims=(0, 1),legend=true)
    end
    gif(anim,"correlation_contour_anim,T="*string(T)*",model="*string(model)*".gif")
end

function correlation_surface_plot(corr, nframe, site_lim,T,model)
    default(legend=false)
    x=y=1:site_lim
    anim = @animate for i=1:n
        f(x, y) = abs((corr[i])[x,y])
        surface(x, y, f,zlim=(0,1),xlabel="sites",ylabel="sites",
             title="animation of correlation matrix over time", c = :blues,clims=(0,1))
    end
    gif(anim,"correlation_surface_plot_anim,T="*string(T)*",model="*string(model)*".gif")
end

function entanglement_animation(SvN, nframe, site_lim, SvN_ylim,T,model)
    anim = @animate for i=1:nframe
        plot(sites[1:site_lim],SvN[i][1:site_lim],ylim=(0,SvN_ylim),
         xlabel="sites",ylabel="entanglement entropy")
    end
    gif(anim,"entanglement_entropy_anim,T="*string(T)*",model="*string(model)*".gif")
end
    

entanglement_animation (generic function with 1 method)